In [11]:
overlay_path = r"../../neorv/rv32i_overlay.bit"
program_name = "blink_all_LEDs"

In [12]:
from pynq.overlay import Overlay
neo_overlay = Overlay(overlay_path)

In [13]:
from pynq import GPIO

neo_reset_n = GPIO(GPIO.get_gpio_pin(0), 'out')
neo_reset_n.write(1)


In [14]:
# Wrtie bin file to memory
little_endian = False
BRAM_data = [] 
with open(program_name + ".bin", "rb") as f:
    addr = 0
    raw_bytes = raw_bytes = f.read(4)
    while raw_bytes != b"":
        if little_endian:
            BRAM_data.append(
                sum([
                    raw_bytes[0] << 3*8,
                    raw_bytes[1] << 2*8,
                    raw_bytes[2] << 1*8,
                    raw_bytes[3] << 0*8,
                ])
            )   
        else:
            BRAM_data.append(
                sum([
                    raw_bytes[3] << 3*8,
                    raw_bytes[2] << 2*8,
                    raw_bytes[1] << 1*8,
                    raw_bytes[0] << 0*8,
                ])
            )

        addr += 4
        raw_bytes = f.read(4)

#for addr, data in enumerate(BRAM_data):#
#    print(hex(4*addr)[2:], hex(data)[2:])

neo_reset_n.write(0)

for addr, word in enumerate(BRAM_data):
    neo_overlay.pynq_BRAM_controller.write(4*addr, word)

neo_reset_n.write(1)

In [15]:
# Neo kickoff

neo_reset_n.write(0)

for _ in range(2**10):
    pass
    
neo_reset_n.write(1)